# Load all the imports!

In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import seaborn as sns;
import string

In [2]:
!pip install transformers

In [3]:
!pip install huggingface_hub

In [4]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227138 sha256=be70d2f546b5c2f7362974106a76da2cc3d286e6f5cb7d5f64be402d8318af18
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [5]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [6]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
import glob
import pandas as pd
import csv
import re
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoTokenizer, AutoModel, AdamW, BertTokenizer, BertForSequenceClassification
import torch

from torch import tensor
from torchmetrics.regression import KendallRankCorrCoef

import fasttext
from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt

import pandas as pd
import json

import random
import numpy as np
import os

In [9]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [10]:
worksheet = gc.open('BEA 2024 Task Data Extended Shuffled').sheet1

rows = worksheet.get_all_records()
print(rows)

[{'ItemNum': 622, 'ItemStem_Text': 'A 27-year-old man comes to the emergency department because of a 3-hour history of pain around his navel. He also has had nausea and one episode of vomiting 1 hour ago. During the past year, he has had multiple episodes of dark stools, which last for 2 to 3 days and resolve spontaneously. He underwent an appendectomy 2 years ago. The patient says that his current symptoms are similar to those he had during the appendicitis episode. His temperature is 39.0Ã‚Â°C (102.2Ã‚Â°F), pulse is 94/min, respirations are 22/min, and blood pressure is 114/78 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 99%. Physical examination shows rebound tenderness localized over the right lower quadrant. Results of laboratory studies are shown: Hemoglobin 12 g/dL Hematocrit 36% Leukocyte count 18,000/mm3 Platelet count 350,000/mm3 Serum Ã‚Â\xa0 Ã¢â‚¬Æ’Urea nitrogen 20 mg/dL Ã¢â‚¬Æ’Creatinine 0.7 mg/dLCT scan of the abdomen shows fat stranding in the mesenter

In [11]:
# Convert to a DataFrame and render.
import pandas as pd

dataset = pd.DataFrame.from_records(rows)
dataset

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,,,,,,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,,,,,,E,Stage of disease,Text,STEP 3,0.50,77.53
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,,,,,,C,Ultrasonography,Text,STEP 2,0.58,65.15
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,,,,,,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,,,,,,B,Chvostek sign,Text,STEP 3,0.22,57.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,631,A 63-year-old man with chronic obstructive pul...,Continuous nebulized albuterol therapy,CT scan of the chest without contrast,Intravenous aminophylline therapy,Pulmonary function testing,Pulmonary rehabilitation,Titrated oxygen therapy,,,,,F,Titrated oxygen therapy,Text,STEP 2,0.38,122.77
462,457,A 58-year-old man comes to the physician becau...,Cremasteric artery,Pampiniform plexus,Testicular artery,Testicular vein,,,,,,,B,Pampiniform plexus,PIX,STEP 1,0.40,79.81
463,114,A 25-year-old man is brought to the office by ...,Advise the parents not to influence their son'...,Arrange for evaluation of their son's competency,Enroll their son in a smoking cessation program,Initiate a family meeting to discuss the paren...,Obtain legal advice regarding guardianship,,,,,,D,Initiate a family meeting to discuss the paren...,Text,STEP 3,0.30,74.61
464,27,A 78-year-old woman is admitted to the hospita...,Her eligibility to receive Medicare,Her history of an abdominal operation,Her history of arthritis,Her suitability for home dialysis,Her willingness to move to the city,,,,,,D,Her suitability for home dialysis,Text,STEP 3,0.16,74.00


# Dataset preprocessing

In [12]:
def transform_column_time(df_column):
  scaler_time = MinMaxScaler()
  scores = np.array(df_column)
  scores = scores.reshape(-1,1)
  scaler_time.fit(scores)
  scores = scaler_time.transform(scores)
  scores = np.float32(scores)
  return (scores, scaler_time)

In [13]:
dataset['Answer__A'] = dataset['Answer__A'].apply(str)
dataset['Answer__B'] = dataset['Answer__B'].apply(str)
dataset['Answer__C'] = dataset['Answer__C'].apply(str)
dataset['Answer__D'] = dataset['Answer__D'].apply(str)
dataset['Answer__E'] = dataset['Answer__E'].apply(str)
dataset['Answer__F'] = dataset['Answer__F'].apply(str)
dataset['Answer__G'] = dataset['Answer__G'].apply(str)
dataset['Answer__H'] = dataset['Answer__H'].apply(str)
dataset['Answer__I'] = dataset['Answer__I'].apply(str)
dataset['Answer__J'] = dataset['Answer__J'].apply(str)
dataset['Answer_Key'] = dataset['Answer_Key'].apply(str)
dataset['Answer_Text'] = dataset['Answer_Text'].apply(str)
# dataset['all_input'] = dataset['all_input'].apply(str)
# dataset['q_answers_input'] = dataset['q_answers_input'].apply(str)
# dataset['answers_input'] = dataset['answers_input'].apply(str)
# dataset['q_a_input'] = dataset['q_a_input'].apply(str)
# dataset['llms_a_input'] = dataset['llms_a_input'].apply(str)
# dataset['q_llms_a_input'] = dataset['q_llms_a_input'].apply(str)

In [14]:
(dataset['Response_Time'], scaler_time) = transform_column_time(dataset['Response_Time'])

In [15]:
fold_1 = dataset[:92]
fold_2 = dataset[93:185]
fold_3 = dataset[186:278]
fold_4 = dataset[279:371]
fold_5 = dataset[372:465]

print(len(fold_1))
print(len(fold_2))
print(len(fold_3))
print(len(fold_4))
print(len(fold_5))

92
92
92
92
93


#Load BERT for classification

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
class CustomBertForSeqClassification(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states=True)
        self.bert.classifier = torch.nn.Linear(768, 1, bias=True)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sigm = self.sigmoid(bert_out.logits)

        return (sigm)

In [18]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [19]:
def k_fold_training(df_train, df_validation):
  print('Number of training sentences: {:,}\n'.format(df_train.shape[0]))
  print('Number of validation sentences: {:,}\n'.format(df_validation.shape[0]))

  ### TRAIN ###
  inputs_train = df_train.ItemStem_Text.values
  labels_train = df_train.Response_Time.values

  ### VALIDATION ###
  inputs_validation = df_validation.ItemStem_Text.values
  labels_validation = df_validation.Response_Time.values

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids_train = []
  attention_masks_train = []
  token_type_ids_train = []

  # For every webpage content.
  for content in inputs_train:
      encoded_dict = tokenizer.encode_plus(
                          content,                   # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 200,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      # Add the encoded sentence to the list.
      input_ids_train.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks_train.append(encoded_dict['attention_mask'])

      token_type_ids_train.append(encoded_dict['token_type_ids'])

  # Convert the lists into tensors.
  input_ids_train = torch.cat(input_ids_train, dim=0)
  attention_masks_train = torch.cat(attention_masks_train, dim=0)
  token_type_ids_train = torch.cat(token_type_ids_train, dim = 0)
  labels_train = torch.tensor(labels_train)

  # # Print sentence 0, now as a list of IDs.
  # print('Original: ', inputs_train[0])
  # print('Token IDs:', input_ids_train[0])


  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids_validation = []
  attention_masks_validation = []
  token_type_ids_validation = []

  # For every webpage content.
  for content in inputs_validation:
      encoded_dict = tokenizer.encode_plus(
                          content,                   # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 200,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      # Add the encoded sentence to the list.
      input_ids_validation.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks_validation.append(encoded_dict['attention_mask'])

      token_type_ids_validation.append(encoded_dict['token_type_ids'])

  # Convert the lists into tensors.
  input_ids_validation = torch.cat(input_ids_validation, dim=0)
  attention_masks_validation = torch.cat(attention_masks_validation, dim=0)
  token_type_ids_validation = torch.cat(token_type_ids_validation, dim=0)
  labels_validation = torch.tensor(labels_validation)

  # Print sentence 0, now as a list of IDs.
  # print('Original: ', inputs_validation[0])
  # print('Token IDs:', input_ids_validation[0])


  # Combine the training inputs into a TensorDataset.
  train_dataset = TensorDataset(input_ids_train, attention_masks_train, token_type_ids_train, labels_train)
  validation_dataset = TensorDataset(input_ids_validation, attention_masks_validation, token_type_ids_validation, labels_validation)


  batch_size = 32

  train_dataloader = DataLoader(
              train_dataset,  # The training samples.
              sampler = RandomSampler(train_dataset), # Select batches randomly
              batch_size = batch_size, # Trains with this batch size.
              num_workers=0
          )

  validation_dataloader = DataLoader(
              validation_dataset, # The validation samples.
              sampler = RandomSampler(validation_dataset), # Pull out batches sequentially.
              batch_size = batch_size, num_workers=0 # Evaluate with this batch size.
          )


  model = CustomBertForSeqClassification()
  # Tell pytorch to run this model on the GPU.
  model.cuda()

  epochs = 10

  # Total number of training steps is [number of batches] x [number of epochs].
  # (Note that this is not the same as the number of training samples).
  total_steps = len(train_dataloader) * epochs

  optimizer = AdamW(model.parameters(),
                    lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  seed_val = 42

  criterion = torch.nn.MSELoss()
  kendall = KendallRankCorrCoef()

  training_stats = []
  total_t0 = time.time()

  # For each epoch...
  for epoch_i in range(0, epochs):

      # ========================================
      #               Training
      # ========================================

      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0
      total_train_kendall_corr = 0
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 500 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)

              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # print(batch[0])
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_input_task_ids = batch[2].to(device)
          b_labels = batch[3].to(device)

          model.zero_grad()

          outputs = model(input_ids=b_input_ids, attention_mask = b_input_mask, token_type_ids = b_input_task_ids)
          outputs = outputs.squeeze(1)
          # print(outputs)
          # print(b_labels)
          loss = criterion(outputs, b_labels)

          total_train_loss += loss.item()

          kendall_corr = kendall(outputs, b_labels)
          # print("Kendall corr: ", kendall_corr)
          total_train_kendall_corr += kendall_corr.item()

          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)
      avg_train_kendall_corr = total_train_kendall_corr / len(train_dataloader)

      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.4f}".format(avg_train_loss))
      print("  Average training Kendall correlation: {0:.4f}".format(avg_train_kendall_corr))
      print("  Training epoch took: {:}".format(training_time))

      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables
      total_eval_loss = 0
      total_eval_kendall_corr = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:

          # Unpack this training batch from our dataloader.
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids
          #   [1]: attention masks
          #   [2]: labels
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_input_task_ids = batch[2].to(device)
          b_labels = batch[3].to(device)

          with torch.no_grad():
              outputs = model(input_ids=b_input_ids, attention_mask = b_input_mask, token_type_ids = b_input_task_ids)

          outputs = outputs.squeeze(1)
          kendall_corr = kendall(outputs, b_labels)

          loss = criterion(outputs, b_labels)
          # Accumulate the validation loss.
          total_eval_loss += loss.item()
          total_eval_kendall_corr += kendall_corr.item()

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      avg_val_kendall_corr = total_eval_kendall_corr / len(validation_dataloader)
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      print("  Validation Loss: {0:.4f}".format(avg_val_loss))
      print("  Validation Kendall correlation: {0:.4f}".format(avg_val_kendall_corr))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Kendall corr.': avg_train_kendall_corr,
              'Valid. Kendall corr.': avg_val_kendall_corr,
              'Training Time': training_time,
              'Validation Time': validation_time,
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  # Display floats with two decimal places.
  # pd.set_option('precision', 6)

  # Create a DataFrame from our training statistics.
  df_stats = pd.DataFrame(data=training_stats)

  # Use the 'epoch' as the row index.
  df_stats = df_stats.set_index('epoch')

  # A hack to force the column headers to wrap.
  #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

  # Display the table.
  print(df_stats)

  return model

# Run 1

Validation = fold_1

Train = fold_2 + fold_3 + fold_4 + fold_5

In [20]:
df_validation = fold_1

df_train = pd.concat([fold_2, fold_3])
df_train = pd.concat([df_train, fold_4])
df_train = pd.concat([df_train, fold_5])

model= k_fold_training(df_train, df_validation)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Number of training sentences: 369

Number of validation sentences: 92



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.0400
  Average training Kendall correlation: 0.0052
  Training epoch took: 0:00:13

Running Validation...
  Validation Loss: 0.0193
  Validation Kendall correlation: 0.1851
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.0197
  Average training Kendall correlation: 0.0721
  Training epoch took: 0:00:11

Running Validation...
  Validation Loss: 0.0180
  Validation Kendall correlation: 0.2834
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.0177
  Average training Kendall correlation: 0.2501
  Training epoch took: 0:00:11

Running Validation...
  Validation Loss: 0.0180
  Validation Kendall correlation: 0.2888
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.0174
  Average training Kendall correlation: 0.2668
  Training epoch took: 0:00:11

Running Validation...
  Vali

# Run 2

Validation = fold_2

Train = fold_1 + fold_3 + fold_4 + fold_5

In [21]:
df_validation = fold_2

df_train =  pd.concat([fold_1, fold_3])
df_train =  pd.concat([df_train, fold_4])
df_train =  pd.concat([df_train, fold_5])

model = k_fold_training(df_train, df_validation)

Number of training sentences: 369

Number of validation sentences: 92



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.0303
  Average training Kendall correlation: -0.0896
  Training epoch took: 0:00:11

Running Validation...
  Validation Loss: 0.0242
  Validation Kendall correlation: -0.1099
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.0200
  Average training Kendall correlation: 0.0320
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0212
  Validation Kendall correlation: 0.1982
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.0181
  Average training Kendall correlation: 0.1168
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0192
  Validation Kendall correlation: 0.3420
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.0161
  Average training Kendall correlation: 0.2536
  Training epoch took: 0:00:12

Running Validation...
  Va

# Run 3

Validation = fold_3

Train = fold_1 + fold_2 + fold_4 + fold_5

In [22]:
df_validation = fold_3

df_train = pd.concat([fold_1, fold_2])
df_train = pd.concat([df_train, fold_4])
df_train = pd.concat([df_train, fold_5])

fine_tuned_model=k_fold_training(df_train, df_validation)

Number of training sentences: 369

Number of validation sentences: 92



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.0277
  Average training Kendall correlation: -0.0587
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0207
  Validation Kendall correlation: 0.3401
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.0193
  Average training Kendall correlation: 0.1326
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0201
  Validation Kendall correlation: 0.3410
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.0180
  Average training Kendall correlation: 0.2112
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0183
  Validation Kendall correlation: 0.3793
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.0147
  Average training Kendall correlation: 0.3635
  Training epoch took: 0:00:12

Running Validation...
  Val

# Run 4

Validation = fold_4

Train = fold_1 + fold_2 + fold_3 + fold_5

In [23]:
df_validation = fold_4

df_train = pd.concat([fold_1, fold_2])
df_train = pd.concat([df_train, fold_3])
df_train = pd.concat([df_train, fold_5])

model = k_fold_training(df_train, df_validation)

Number of training sentences: 369

Number of validation sentences: 92



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.0227
  Average training Kendall correlation: 0.0972
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0189
  Validation Kendall correlation: 0.2734
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.0189
  Average training Kendall correlation: 0.2070
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0155
  Validation Kendall correlation: 0.2895
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.0169
  Average training Kendall correlation: 0.2779
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0140
  Validation Kendall correlation: 0.3795
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.0147
  Average training Kendall correlation: 0.3495
  Training epoch took: 0:00:12

Running Validation...
  Vali

# Run 5

Validation = fold_5

Train = fold_1 + fold_2 + fold_3 + fold_4

In [24]:
df_validation = fold_5

df_train = pd.concat([fold_1, fold_2])
df_train = pd.concat([df_train, fold_3])
df_train = pd.concat([df_train, fold_4])

model = k_fold_training(df_train, df_validation)

Number of training sentences: 368

Number of validation sentences: 93



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.0538
  Average training Kendall correlation: 0.0038
  Training epoch took: 0:00:11

Running Validation...
  Validation Loss: 0.0157
  Validation Kendall correlation: 0.2601
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.0197
  Average training Kendall correlation: 0.1694
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0133
  Validation Kendall correlation: 0.3603
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.0181
  Average training Kendall correlation: 0.2653
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.0131
  Validation Kendall correlation: 0.3118
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.0172
  Average training Kendall correlation: 0.3149
  Training epoch took: 0:00:12

Running Validation...
  Vali

In [25]:
df_train = pd.concat([fold_1, fold_2])
df_train = pd.concat([df_train, fold_3])
df_train = pd.concat([df_train, fold_4])
df_train = pd.concat([df_train, fold_5])

print('Number of training sentences: {:,}\n'.format(df_train.shape[0]))

### TRAIN ###
inputs_train = df_train.ItemStem_Text
labels_train = df_train.Response_Time.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_train = []
attention_masks_train = []
token_type_ids_train = []

# For every webpage content.
for content in inputs_train:
    encoded_dict = tokenizer.encode_plus(
                        content,                   # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                  )
    # Add the encoded sentence to the list.
    input_ids_train.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict['attention_mask'])

    token_type_ids_train.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
token_type_ids_train = torch.cat(token_type_ids_train, dim = 0)
labels_train = torch.tensor(labels_train)

# # Print sentence 0, now as a list of IDs.
# print('Original: ', inputs_train[0])
# print('Token IDs:', input_ids_train[0])


# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, token_type_ids_train, labels_train)

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size, # Trains with this batch size.
            num_workers=0
        )



model = CustomBertForSeqClassification()
# Tell pytorch to run this model on the GPU.
model.cuda()

epochs = 15

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
seed_val = 42

criterion = torch.nn.MSELoss()
kendall = KendallRankCorrCoef()

training_stats = []
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    total_train_kendall_corr = 0
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 500 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # print(batch[0])
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_input_task_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()

        outputs = model(input_ids=b_input_ids, attention_mask = b_input_mask, token_type_ids = b_input_task_ids)
        outputs = outputs.squeeze(1)
        # print(outputs)
        # print(b_labels)
        loss = criterion(outputs, b_labels)

        total_train_loss += loss.item()

        kendall_corr = kendall(outputs, b_labels)
        # print("Kendall corr: ", kendall_corr)
        total_train_kendall_corr += kendall_corr.item()

        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_kendall_corr = total_train_kendall_corr / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Average training Kendall correlation: {0:.4f}".format(avg_train_kendall_corr))
    print("  Training epoch took: {:}".format(training_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Kendall corr.': avg_train_kendall_corr,
            'Training Time': training_time,
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
# Display floats with two decimal places.
# pd.set_option('precision', 6)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
print(df_stats)

Number of training sentences: 461



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use th


======== Epoch 1 / 15 ========
Training...

  Average training loss: 0.0420
  Average training Kendall correlation: 0.0120
  Training epoch took: 0:00:14

======== Epoch 2 / 15 ========
Training...

  Average training loss: 0.0196
  Average training Kendall correlation: 0.0430
  Training epoch took: 0:00:15

======== Epoch 3 / 15 ========
Training...

  Average training loss: 0.0183
  Average training Kendall correlation: 0.1710
  Training epoch took: 0:00:15

======== Epoch 4 / 15 ========
Training...

  Average training loss: 0.0160
  Average training Kendall correlation: 0.3110
  Training epoch took: 0:00:14

======== Epoch 5 / 15 ========
Training...

  Average training loss: 0.0159
  Average training Kendall correlation: 0.3078
  Training epoch took: 0:00:14

======== Epoch 6 / 15 ========
Training...

  Average training loss: 0.0148
  Average training Kendall correlation: 0.3488
  Training epoch took: 0:00:14

======== Epoch 7 / 15 ========
Training...

  Average training loss: 

# Evaluate on test set


In [152]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [153]:
worksheet = gc.open('BEA 2024 Test Data Extended').sheet1

rows = worksheet.get_all_records()
print(rows)

[{'ItemNum': 552, 'ItemStem_Text': 'A 78-year-old woman undergoes an outpatient laparoscopic cholecystectomy. During the procedure, ligation of the cystic artery is planned. It is most appropriate to ligate this artery at its junction with which of the following arteries?', 'Answer__A': 'Gastroduodenal', 'Answer__B': 'Left gastric', 'Answer__C': 'Left hepatic', 'Answer__D': 'Right gastric', 'Answer__E': 'Right hepatic', 'Answer__F': '', 'Answer__G': '', 'Answer__H': '', 'Answer__I': '', 'Answer__J': '', 'Answer_Key': 'E', 'Answer_Text': 'Right hepatic', 'ItemType': 'Text', 'EXAM': 'STEP 1', 'Difficulty': 0.52, 'Response_Time': 75.21}, {'ItemNum': 16, 'ItemStem_Text': 'A 48-year-old man dies suddenly of a cardiac arrhythmia. Six weeks ago, he was resuscitated from a cardiac arrest caused by coronary artery disease and developed oliguric renal failure. Five weeks ago (1 week after being resuscitated), his serum urea nitrogen (BUN) concentration was 40 mg/dL, his serum creatinine concentr

In [154]:
# Convert to a DataFrame and render.
import pandas as pd

dataset = pd.DataFrame.from_records(rows)
dataset

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,552,A 78-year-old woman undergoes an outpatient la...,Gastroduodenal,Left gastric,Left hepatic,Right gastric,Right hepatic,,,,,,E,Right hepatic,Text,STEP 1,0.52,75.21
1,16,A 48-year-old man dies suddenly of a cardiac a...,Acute inflammation,Fibrinous exudate,Fibrous scar,Granulation tissue,Normal kidney,,,,,,E,Normal kidney,Text,STEP 1,0.70,80.88
2,441,A study is being conducted to assess mesotheli...,Address confounding,Demonstrate causality,Minimize ascertainment bias,Reduce recall bias,,,,,,,C,Minimize ascertainment bias,Text,STEP 3,0.84,88.45
3,219,A 68-year-old man comes to the office because ...,Age,Gender,History of osteoarthritis,Previous occupation,Use of pravastatin,,,,,,A,Age,Text,STEP 3,1.02,127.30
4,600,A 20-year-old woman is brought to the urgent c...,Diet,Family illness,Recent travel,Trauma,Unintended weight loss,,,,,,D,Trauma,Text,STEP 1,0.96,88.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,612,A 45-year-old man is brought to the clinic by ...,Atrophy,Decreased visual acuity,Loss of proprioception,Presence of palmomental reflex,Ptosis,,,,,,A,Atrophy,Text,STEP 1,0.22,95.17
197,315,A 64-year-old man with non-Hodgkin lymphoma co...,Bleomycin,Cyclophosphamide,Cytarabine,Doxorubicin,Fluorouracil,Methotrexate,Vincristine,,,,G,Vincristine,Text,STEP 1,0.14,42.72
198,509,A 9-month-old boy is brought to the office by ...,Abdominal ultrasonography,AntiÃ¢â‚¬â€œSaccharomyces cerevisiae antibody ...,Nitroblue tetrazolium testing,Rectal swab for group B streptococcus,Skeletal survey,Stool culture for Salmonella enteritidis,,,,,E,Skeletal survey,Text,STEP 2,0.94,122.73
199,550,A 32-year-old man comes to the office because ...,Collagen,Double-stranded DNA,Nucleolar protein,Phospholipid,Proteins in neutrophil cytoplasm,,,,,,A,Collagen,Text,STEP 1,0.30,86.13


In [162]:
scores = np.array(dataset['Response_Time'])
scores = scores.reshape(-1,1)
scores = scaler_time.transform(scores)
dataset['Response_Time'] = np.float32(scores)

In [170]:
labels_test = dataset.Response_Time.values
labels_test = torch.tensor(labels_test)

In [171]:
dataset

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,552,A 78-year-old woman undergoes an outpatient la...,Gastroduodenal,Left gastric,Left hepatic,Right gastric,Right hepatic,,,,,,E,Right hepatic,Text,STEP 1,0.52,0.238307
1,16,A 48-year-old man dies suddenly of a cardiac a...,Acute inflammation,Fibrinous exudate,Fibrous scar,Granulation tissue,Normal kidney,,,,,,E,Normal kidney,Text,STEP 1,0.70,0.266311
2,441,A study is being conducted to assess mesotheli...,Address confounding,Demonstrate causality,Minimize ascertainment bias,Reduce recall bias,,,,,,,C,Minimize ascertainment bias,Text,STEP 3,0.84,0.303699
3,219,A 68-year-old man comes to the office because ...,Age,Gender,History of osteoarthritis,Previous occupation,Use of pravastatin,,,,,,A,Age,Text,STEP 3,1.02,0.495580
4,600,A 20-year-old woman is brought to the urgent c...,Diet,Family illness,Recent travel,Trauma,Unintended weight loss,,,,,,D,Trauma,Text,STEP 1,0.96,0.304292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,612,A 45-year-old man is brought to the clinic by ...,Atrophy,Decreased visual acuity,Loss of proprioception,Presence of palmomental reflex,Ptosis,,,,,,A,Atrophy,Text,STEP 1,0.22,0.336889
197,315,A 64-year-old man with non-Hodgkin lymphoma co...,Bleomycin,Cyclophosphamide,Cytarabine,Doxorubicin,Fluorouracil,Methotrexate,Vincristine,,,,G,Vincristine,Text,STEP 1,0.14,0.077839
198,509,A 9-month-old boy is brought to the office by ...,Abdominal ultrasonography,AntiÃ¢â‚¬â€œSaccharomyces cerevisiae antibody ...,Nitroblue tetrazolium testing,Rectal swab for group B streptococcus,Skeletal survey,Stool culture for Salmonella enteritidis,,,,,E,Skeletal survey,Text,STEP 2,0.94,0.473008
199,550,A 32-year-old man comes to the office because ...,Collagen,Double-stranded DNA,Nucleolar protein,Phospholipid,Proteins in neutrophil cytoplasm,,,,,,A,Collagen,Text,STEP 1,0.30,0.292241


In [172]:
dataset['Answer__A'] = dataset['Answer__A'].apply(str)
dataset['Answer__B'] = dataset['Answer__B'].apply(str)
dataset['Answer__C'] = dataset['Answer__C'].apply(str)
dataset['Answer__D'] = dataset['Answer__D'].apply(str)
dataset['Answer__E'] = dataset['Answer__E'].apply(str)
dataset['Answer__F'] = dataset['Answer__F'].apply(str)
dataset['Answer__G'] = dataset['Answer__G'].apply(str)
dataset['Answer__H'] = dataset['Answer__H'].apply(str)
dataset['Answer__I'] = dataset['Answer__I'].apply(str)
dataset['Answer__J'] = dataset['Answer__J'].apply(str)
dataset['Answer_Key'] = dataset['Answer_Key'].apply(str)
dataset['Answer_Text'] = dataset['Answer_Text'].apply(str)
# dataset['all_input'] = dataset['all_input'].apply(str)
# dataset['q_answers_input'] = dataset['q_answers_input'].apply(str)
# dataset['answers_input'] = dataset['answers_input'].apply(str)
# dataset['q_a_input'] = dataset['q_a_input'].apply(str)
# dataset['llms_a_input'] = dataset['llms_a_input'].apply(str)
# dataset['q_llms_a_input'] = dataset['q_llms_a_input'].apply(str)

In [173]:
inputs_test = dataset.ItemStem_Text.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_test = []
attention_masks_test = []
token_type_ids_test = []

# For every webpage content.
for content in inputs_test:
    encoded_dict = tokenizer.encode_plus(
                        content,                   # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                  )

    # Add the encoded sentence to the list.
    input_ids_test.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])

    token_type_ids_test.append(encoded_dict['token_type_ids'])

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
token_type_ids_test = torch.cat(token_type_ids_test, dim=0)
item_nums_test = torch.tensor(dataset.ItemNum.values)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, token_type_ids_test, item_nums_test, labels_test)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [174]:
test_dataloader = DataLoader(test_dataset,
                             sampler=SequentialSampler(test_dataset),
                             batch_size=32)

In [175]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids_test)))

# Put model in evaluation mode
model.eval()

# Tracking variables
item_nums = []
predictions = []

total_test_loss = 0
total_test_kendall = 0

criterion = torch.nn.MSELoss()
kendall = KendallRankCorrCoef()

# Predict
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_input_task_ids, b_item_nums, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
    outputs = model(input_ids=b_input_ids, attention_mask = b_input_mask, token_type_ids = b_input_task_ids)

  loss = criterion(outputs, b_labels)

  total_test_loss += loss.item()

  kendall_corr = kendall(outputs.flatten(), b_labels)
  total_test_kendall += kendall_corr

  predicted_labels = outputs.detach().cpu().numpy()
  current_item_nums = b_item_nums.to('cpu').numpy()

  # Store predictions and true labels
  predictions = np.append(predictions, predicted_labels)
  item_nums = np.append(item_nums, current_item_nums)

print('    DONE.')
# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)
avg_test_kendall_corr = total_test_kendall / len(test_dataloader)
print("  Validation Loss: {0:.4f}".format(avg_test_loss))
print("  Validation Kendall correlation: {0:.4f}".format(avg_test_kendall_corr))

Predicting labels for 201 test sentences...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


    DONE.
  Validation Loss: 0.0316
  Validation Kendall correlation: 0.3524


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [176]:
predictions = predictions.reshape(-1,1)
predictions = scaler_time.inverse_transform(predictions)

In [177]:
predictions = predictions.flatten()

In [201]:
item_nums = item_nums.astype(int)

In [217]:
d = {'ItemNum': item_nums, 'Prediction': predictions}
df = pd.DataFrame(data=d)

# Merge the dataset to include 'Response_Time' column in df
df = df.merge(dataset[['ItemNum', 'Response_Time']], on='ItemNum')

In [209]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE
rmse = mean_squared_error(df['Response_Time'], predictions, squared=False)

print(f"RMSE: {rmse}")

RMSE: 25.3738022371596


In [210]:
df.to_csv("Response_time_predictions_BERT.csv")